In [2]:
import os  # for accessing the file system
from itertools import product  # for efficient looping process

import numpy as np  # for linear algebra
import pandas as pd  # for data analysis

# for image processing
import PIL

# for deep learning process
import tensorflow as tf
from keras.preprocessing import image
from keras.utils import image_dataset_from_directory
from PIL import Image
from tensorflow import keras
from tensorflow.keras import Sequential, layers


# Find the count of the files in the datasets directory

In [19]:
set_list = ["test_set", "training_set"]
cats_dogs = ["cats", "dogs"]
folderpath = [
    os.path.abspath(f"datasets/{sets}/{animals}")
    for sets, animals in product(set_list, cats_dogs)
]

count = sum(len(os.listdir(folderpath[i])) for i in range(len(folderpath)))
print(f"Total images found: {count} files belonging")

Total images found: 10028 files belonging


In [20]:
train_ds = image_dataset_from_directory(
    "datasets/training_set",
    labels="inferred",
    label_mode="binary",
    color_mode="rgb",
    batch_size=32,
    image_size=(64, 64),
    interpolation="bilinear",
)
test_ds = image_dataset_from_directory(
    "datasets/test_set",
    labels="inferred",
    label_mode="binary",
    color_mode="rgb",
    batch_size=32,
    image_size=(64, 64),
    interpolation="bilinear",
)
print(train_ds, test_ds)

Found 8005 files belonging to 2 classes.
Found 2023 files belonging to 2 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))> <BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


# Create the model using the Sequential API